## Leitura e tratamento dos dados

### Importação de Bibliotecas

In [26]:
import pandas as pd

path_dados_vinho = "../../dados/vinho/"

### Exportação

In [27]:
df_exp_espumantes = pd.read_csv(f"{path_dados_vinho}ExpEspumantes.csv", sep=";")
df_exp_suco = pd.read_csv(f"{path_dados_vinho}ExpSuco.csv", sep=";")
df_exp_uva = pd.read_csv(f"{path_dados_vinho}ExpUva.csv", sep=";")
df_exp_vinho = pd.read_csv(f"{path_dados_vinho}ExpVinho.csv", sep=";")

### Comércio

In [62]:
df_comercio = pd.read_csv(f"{path_dados_vinho}Comercio.csv", sep=";", header=None)

df_comercio.columns = ['ID', 'Categoria', 'Subcategoria'] + [str(ano) for ano in range(1970, 2022)]

def categorizar(cat):
    prefixo = cat.split('_')[0]
    resto = '_'.join(cat.split('_')[1:])
    
    if prefixo == 'vm':
        return 'VINHO DE MESA_' + resto
    elif prefixo == 've':
        return 'VINHO ESPECIAL_' + resto
    elif prefixo == 'es':
        return 'ESPUMANTES_' + resto
    elif prefixo == 'su':
        return 'SUCO DE UVAS_' + resto
    elif prefixo == 'ou':
        return 'OUTROS PRODUTOS COMERCIALIZADOS_' + resto
    else:
        return cat

df_comercio['Categoria'] = df_comercio['Categoria'].apply(categorizar)

# Extrair categorias principais e subcategorias
df_comercio['Categoria_Split'] = df_comercio['Categoria'].str.split('_')
df_comercio['Categoria_Principal'] = df_comercio['Categoria_Split'].apply(lambda x: x[0] if x else '')
df_comercio['Subcategoria'] = df_comercio['Categoria_Split'].apply(lambda x: '_'.join(x[1:]) if len(x) > 1 else '')
df_comercio['Subcategoria'] = df_comercio.apply(lambda row: row['Categoria_Principal'] if row['Subcategoria'] == '' else row['Subcategoria'], axis=1)

df_comercio = df_comercio.drop('Categoria_Split', axis=1)

### Importação

In [29]:
df_imp_espumantes = pd.read_csv(f"{path_dados_vinho}ImpEspumantes.csv", sep=";")
df_imp_frescas = pd.read_csv(f"{path_dados_vinho}ImpFrescas.csv", sep=";")
df_imp_passas = pd.read_csv(f"{path_dados_vinho}ImpPassas.csv", sep=";")
df_imp_suco = pd.read_csv(f"{path_dados_vinho}ImpSuco.csv", sep=";")
df_imp_vinhos = pd.read_csv(f"{path_dados_vinho}ImpVinhos.csv", sep=";")

### Processamento

In [40]:
df_proc_americanas = pd.read_csv(f"{path_dados_vinho}ProcessaAmericanas.csv", sep=";")
df_proc_mesa = pd.read_csv(f"{path_dados_vinho}ProcessaMesa.csv", sep=";")
df_proc_viniferas = pd.read_csv(f"{path_dados_vinho}ProcessaViniferas.csv", sep="\t")

### Produção

In [31]:
df_produção = pd.read_csv(f"{path_dados_vinho}Producao.csv", sep=";")

## Tratamento das Bases

In [63]:
df_comercio.head()

,ID,Categoria,Subcategoria,1970,1971,1972,1973,1974,1975,1976,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Categoria_Principal
0,1,VINHO DE MESA,VINHO DE MESA,98327606,114399031,118377367,116617910,94173324,108031792,139238614,...,221590810,206404427,209198468,166769622,176059959,177186273,180446489,215557931,210012238,VINHO DE MESA
1,2,VINHO DE MESA_Tinto,Tinto,83300735,98522869,101167932,98196747,77167303,91528090,116407222,...,188033494,178250072,182028785,146646365,154309442,155115499,158519218,189573423,185653678,VINHO DE MESA
2,3,VINHO DE MESA_Rosado,Rosado,107681,542274,7770851,8425617,8891367,7261777,11748047,...,1777648,1419855,1409002,1391942,1097426,1972944,1265435,1394901,1931606,VINHO DE MESA
3,4,VINHO DE MESA_Branco,Branco,14919190,15333888,9438584,9995546,8114654,9241925,11083345,...,31779668,26734500,25760681,18731315,20653091,20097830,20661836,24589607,22426954,VINHO DE MESA
4,5,VINHO FINO DE MESA,VINHO FINO DE MESA,4430629,4840369,5602091,7202830,7571802,8848303,14095648,...,27912934,20424983,20141631,19630158,15874354,14826143,15684588,24310834,27080445,VINHO FINO DE MESA
